# Py City Schools Analysis

In [533]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

In [604]:
# Calculate the total number of unique schools
school_count = len(school_data_complete["School ID"].unique())
print(school_count)
#I could have also written school_count = len(school_data_complete["school_name"].unique()) and returned the same result

15


In [535]:
# Calculate the total number of students
student_count = len(school_data_complete["Student ID"].unique())
student_count_formatted = ('{:,}'.format(student_count))
print(student_count_formatted)
# Why did I use ID number despite starter code suggesting I use student_name column?
# Well, at first I used the student names. My code looked like
# student_count = len(school_data_complete["student_name"].unique())
# This was per the suggestion in the starter code.
# The resultant count was 32715 students.
# When I tried to calculate the percentage of students who passed reading using student_name to calculate student_count,
# My result was 102%. Obviously incorrect.
# Then, I realized that it's possible for some students to have same first and last name.
# I could have written
# student_count = len(school_data_complete["student_name"],
# But I believe that if I were really working for a school district,
# the best way to keep track of students would be by using ID numbers rather than names.

39,170


In [536]:
# Calculate the total budget
total_budget = sum(school_data_complete["budget"].unique())
total_budget_rounded = round(total_budget, 2)
total_budget_formatted = ('${:,}'.format(total_budget_rounded))
print(total_budget_formatted)

$24,649,428


In [537]:
# Calculate the average (mean) math score
average_math_score = (school_data_complete["math_score"].mean())
m_score_rounded = round(average_math_score, 2)
print(m_score_rounded)

78.99


In [538]:
# Calculate the average (mean) reading score
average_reading_score = (school_data_complete["reading_score"].mean())
r_score_rounded = round(average_reading_score, 2)
print(r_score_rounded)

81.88


In [539]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["Student ID"]
passing_math_percentage = float(passing_math_count) / float(student_count) * 100
passing_math_rounded = round(passing_math_percentage, 2)
percent_passing_math_formatted = ('{}%'.format(passing_math_rounded))
print(percent_passing_math_formatted)

74.98%


In [636]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >=70)].count()["Student ID"]
passing_reading_percentage = (passing_reading_count) / (student_count) * 100
passing_reading_rounded = round(passing_reading_percentage, 2)
percent_passing_reading_formatted = ('{}%'.format(passing_reading_rounded))
print(percent_passing_reading_formatted)

85.81%


In [541]:
# Use the following to calculate the percentage of students that passed math and reading
passing_both_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["Student ID"]
overall_passing_rate = passing_both_count / float(student_count) * 100
overall_passing_rate_rounded = round(overall_passing_rate, 2)
overall_passing_rate_formatted = ('{}%'.format(overall_passing_rate_rounded))
print(overall_passing_rate_formatted)

65.17%


In [542]:
district_dataframe_summary = {"Total Schools": [school_count],
                              "Total Students": [student_count_formatted],
                              "Total Budget": [total_budget_formatted],
                              "Average Math Score": [m_score_rounded],
                              "Average Reading Score": [r_score_rounded],
                              "% Passing Math": [percent_passing_math_formatted],
                              "% Passing Reading": [percent_passing_reading_formatted],
                              "% Overall Passing": [overall_passing_rate_formatted]}
df = pd.DataFrame(district_dataframe_summary)
display(df)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,65.17%


# School Summary

In [700]:
# Setting the variables for the school summary dataframe
school_name = (school_data_complete["school_name"].unique())
school_type = school_data_complete.groupby(["School ID"])["type"]
indv_school_size = (school_data_complete["size"].unique())
indv_budget = (school_data_complete["budget"].unique())
budget_per_stu = (indv_budget / indv_school_size)
average_math_score = school_data_complete.groupby(["School ID"])["math_score"].mean()
ams_format = round(average_math_score, 2)
average_reading_score = school_data_complete.groupby(["School ID"])["reading_score"].mean()
ars_format = round(average_reading_score, 2)

print(school_type)

In [674]:
# Now I have to calculate what percentage of students passed math per school
# I need to know how many students go a score of 70 or greater per school
# Then I need to do the formula (passing studets / student body) * 100
passed_math = school_data_complete["math_score"] >= 70
passed_math_per_school = passed_math.groupby(school_data_complete["School ID"]).sum()
percent_passed_math = round(((passed_math_per_school / indv_school_size) * 100), 2)

In [669]:
#Now I will do the same for the reading scores
passed_reading = school_data_complete["reading_score"] >= 70
passed_reading_per_school = passed_reading.groupby(school_data_complete["School ID"]).sum()
percent_passed_reading = round(((passed_reading_per_school / indv_school_size) * 100), 2)

In [683]:
#overall passing % per school
overall_pass = (passed_math_per_school) & (passed_reading_per_school)
overall_pass_percent = round(((overall_pass / indv_school_size) * 100), 2)

In [739]:
school_summary = {"School Name": [school_name[0], school_name[1], school_name[2], school_name[3], school_name[4], school_name[5], school_name[6], school_name[7], school_name[8], school_name[9], school_name[10], school_name[11], school_name[12], school_name[13], school_name[14]],
                  "School Type": [school_type[0], school_type[1], school_type[2], school_type[3], school_type[4], school_type[5], school_type[6], school_type[7], school_type[8], school_type[9], school_type[10], school_type[11], school_type[12], school_type[13], school_type[14]],
                  "Total Students": [indv_school_size[0], indv_school_size[1], indv_school_size[2], indv_school_size[3], indv_school_size[4], indv_school_size[5], indv_school_size[6], indv_school_size[7], indv_school_size[8], indv_school_size[9], indv_school_size[10], indv_school_size[11], indv_school_size[12], indv_school_size[13], indv_school_size[14]],
                  "Total School Budget": [('${}'.format(indv_budget[0])), ('${}'.format(indv_budget[1])), ('${}'.format(indv_budget[2])), ('${}'.format(indv_budget[3])), ('${}'.format(indv_budget[4])), ('${}'.format(indv_budget[5])), ('${}'.format(indv_budget[6])), ('${}'.format(indv_budget[7])), ('${}'.format(indv_budget[8])), ('${}'.format(indv_budget[9])), ('${}'.format(indv_budget[10])), ('${}'.format(indv_budget[11])), ('${}'.format(indv_budget[12])), ('${}'.format(indv_budget[13])), ('${}'.format(indv_budget[14]))],
                  "Budget Per Student": [('${}'.format(budget_per_stu[0])), ('${}'.format(budget_per_stu[1])), ('${}'.format(budget_per_stu[2])), ('${}'.format(budget_per_stu[3])), ('${}'.format(budget_per_stu[4])), ('${}'.format(budget_per_stu[5])), ('${}'.format(budget_per_stu[6])), ('${}'.format(budget_per_stu[7])), ('${}'.format(budget_per_stu[8])), ('${}'.format(budget_per_stu[9])), ('${}'.format(budget_per_stu[10])), ('${}'.format(budget_per_stu[11])), ('${}'.format(budget_per_stu[12])), ('${}'.format(budget_per_stu[13])), ('${}'.format(budget_per_stu[14]))], 
                  "Average Math Score": [ams_format[0], ams_format[1], ams_format[2], ams_format[3], ams_format[4], ams_format[5], ams_format[6], ams_format[7], ams_format[8], ams_format[9], ams_format[10], ams_format[11], ams_format[12], ams_format[13], ams_format[14]],
                  "Average Reading Score": [ars_format[0], ars_format[1], ars_format[2], ars_format[3], ars_format[4], ars_format[5], ars_format[6], ars_format[7], ars_format[8], ars_format[9], ars_format[10], ars_format[11], ars_format[12], ars_format[13], ars_format[14]],
                  "% Passing Math": [percent_passed_math[0], percent_passed_math[1], percent_passed_math[2], percent_passed_math[3], percent_passed_math[4], percent_passed_math[5], percent_passed_math[6], percent_passed_math[7], percent_passed_math[8], percent_passed_math[9], percent_passed_math[10], percent_passed_math[11], percent_passed_math[12], percent_passed_math[13], percent_passed_math[14]], 
                  "% Passing Reading": [percent_passed_reading[0], percent_passed_reading[1], percent_passed_reading[2], percent_passed_reading[3], percent_passed_reading[4], percent_passed_reading[5], percent_passed_reading[6], percent_passed_reading[7], percent_passed_reading[8], percent_passed_reading[9], percent_passed_reading[10], percent_passed_reading[11], percent_passed_reading[12], percent_passed_reading[13], percent_passed_reading[14]],
                  "% Overall Passing": [overall_pass_percent[0], overall_pass_percent[1], overall_pass_percent[2], overall_pass_percent[3], overall_pass_percent[4], overall_pass_percent[5], overall_pass_percent[6], overall_pass_percent[7], overall_pass_percent[8], overall_pass_percent[9], overall_pass_percent[10], overall_pass_percent[11], overall_pass_percent[12], overall_pass_percent[13], overall_pass_percent[14]],}
per_school_summary = pd.DataFrame(school_summary)
per_school_summary.set_index("School Name", inplace=True)
display(per_school_summary)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Huang High School,District,2917,$1910635,$655.0,76.63,81.18,65.68,81.32,11.11
Figueroa High School,District,2949,$1884411,$639.0,76.71,81.16,65.99,80.74,8.95
Shelton High School,Charter,1761,$1056600,$600.0,83.36,83.73,93.87,95.85,88.13
Hernandez High School,District,4635,$3022020,$652.0,77.29,80.93,66.75,80.86,66.36
Griffin High School,Charter,1468,$917500,$625.0,83.35,83.82,93.39,97.14,88.42
Wilson High School,Charter,2283,$1319574,$578.0,83.27,83.99,93.87,96.54,90.93
Cabrera High School,Charter,1858,$1081356,$582.0,83.06,83.98,94.13,97.04,82.72
Bailey High School,District,4976,$3124928,$628.0,77.05,81.03,66.68,81.93,66.32
Holden High School,Charter,427,$248087,$581.0,83.80,83.81,92.51,96.25,92.51


# Highest-Performing Schools (by % Overall Passing)

In [742]:
top_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Pena High School,Charter,962,$585858,$609.0,83.84,84.04,94.59,95.95,94.18
Holden High School,Charter,427,$248087,$581.0,83.80,83.81,92.51,96.25,92.51
Wright High School,Charter,1800,$1049400,$583.0,83.68,83.96,93.33,96.61,92.44
Wilson High School,Charter,2283,$1319574,$578.0,83.27,83.99,93.87,96.54,90.93
Griffin High School,Charter,1468,$917500,$625.0,83.35,83.82,93.39,97.14,88.42


# Bottom Performing Schools (By % Overall Passing)

In [743]:
bottom_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=True)
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Ford High School,District,2739,$1763916,$644.0,77.10,80.75,68.31,79.30,2.77
Figueroa High School,District,2949,$1884411,$639.0,76.71,81.16,65.99,80.74,8.95
Huang High School,District,2917,$1910635,$655.0,76.63,81.18,65.68,81.32,11.11
Rodriguez High School,District,3999,$2547363,$637.0,76.84,80.74,66.37,80.22,51.41
Johnson High School,District,4761,$3094650,$650.0,77.07,80.97,66.06,81.22,64.71


# Math Scores By Grade

In [744]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby("school_name")["math_score"].mean()
tenth_grader_math_scores = tenth_graders.groupby("school_name")["math_score"].mean()
eleventh_grader_math_scores = eleventh_graders.groupby("school_name")["math_score"].mean()
twelfth_grader_math_scores = twelfth_graders.groupby("school_name")["math_score"].mean()

math_scores_by_grade = pd.DataFrame({
    "9th": ninth_grade_math_scores,
    "10th": tenth_grader_math_scores,
    "11th": eleventh_grader_math_scores,
    "12th": twelfth_grader_math_scores,
})
math_scores_by_grade.index.name = None
display(math_scores_by_grade)

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Reading Score by Grade

In [745]:
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby("school_name")["reading_score"].mean()
tenth_grader_reading_scores = tenth_graders.groupby("school_name")["reading_score"].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby("school_name")["reading_score"].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby("school_name")["reading_score"].mean()

# Combine each of the scores above into a single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({
    "9th": ninth_grade_reading_scores,
    "10th": tenth_grader_reading_scores,
    "11th": eleventh_grader_reading_scores,
    "12th": twelfth_grader_reading_scores,
})
reading_scores_by_grade.index.name = None
display(reading_scores_by_grade)

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [735]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [752]:
# Create a copy of the school summary since it has the "Per Student Budget" 
per_school_summary["Budget Per Student"] = per_school_summary["Budget Per Student"].replace('[\$,]', '', regex=True).astype(float)
school_spending_df = per_school_summary.copy()

In [753]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Budget Per Student"], bins=spending_bins, labels=labels)
display(school_spending_df)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
School Name,,,,,,,,,,
Huang High School,District,2917,$1910635,655.0,76.63,81.18,65.68,81.32,11.11,$645-680
Figueroa High School,District,2949,$1884411,639.0,76.71,81.16,65.99,80.74,8.95,$630-645
Shelton High School,Charter,1761,$1056600,600.0,83.36,83.73,93.87,95.85,88.13,$585-630
Hernandez High School,District,4635,$3022020,652.0,77.29,80.93,66.75,80.86,66.36,$645-680
Griffin High School,Charter,1468,$917500,625.0,83.35,83.82,93.39,97.14,88.42,$585-630
Wilson High School,Charter,2283,$1319574,578.0,83.27,83.99,93.87,96.54,90.93,<$585
Cabrera High School,Charter,1858,$1081356,582.0,83.06,83.98,94.13,97.04,82.72,<$585
Bailey High School,District,4976,$3124928,628.0,77.05,81.03,66.68,81.93,66.32,$585-630
Holden High School,Charter,427,$248087,581.0,83.80,83.81,92.51,96.25,92.51,<$585


In [754]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [756]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
})

# Display results
display(spending_summary)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.452500,83.935000,93.460000,96.610000,89.650000
$585-630,81.900000,83.155000,87.132500,92.717500,84.262500
$630-645,78.517500,81.625000,73.485000,84.392500,32.250000
$645-680,76.996667,81.026667,66.163333,81.133333,47.393333


# Scores by School Size

In [762]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [764]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], bins=size_bins, labels=labels)
display(per_school_summary)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
School Name,,,,,,,,,,
Huang High School,District,2917,$1910635,655.0,76.63,81.18,65.68,81.32,11.11,Large (2000-5000)
Figueroa High School,District,2949,$1884411,639.0,76.71,81.16,65.99,80.74,8.95,Large (2000-5000)
Shelton High School,Charter,1761,$1056600,600.0,83.36,83.73,93.87,95.85,88.13,Medium (1000-2000)
Hernandez High School,District,4635,$3022020,652.0,77.29,80.93,66.75,80.86,66.36,Large (2000-5000)
Griffin High School,Charter,1468,$917500,625.0,83.35,83.82,93.39,97.14,88.42,Medium (1000-2000)
Wilson High School,Charter,2283,$1319574,578.0,83.27,83.99,93.87,96.54,90.93,Large (2000-5000)
Cabrera High School,Charter,1858,$1081356,582.0,83.06,83.98,94.13,97.04,82.72,Medium (1000-2000)
Bailey High School,District,4976,$3124928,628.0,77.05,81.03,66.68,81.93,66.32,Large (2000-5000)
Holden High School,Charter,427,$248087,581.0,83.80,83.81,92.51,96.25,92.51,Small (<1000)


In [768]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [769]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": size_overall_passing
})
# Display results
display(size_summary)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.820,83.92500,93.55000,96.10000,93.345
Medium (1000-2000),83.374,83.86800,93.59800,96.79000,83.516
Large (2000-5000),77.745,81.34375,69.96375,82.76625,45.320


# Scores by School Type

In [770]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [773]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({
    "Average Math Score": average_math_score_by_type,
    "Average Reading Score": average_reading_score_by_type,
    "% Passing Math": average_percent_passing_math_by_type,
    "% Passing Reading": average_percent_passing_reading_by_type,
    "% Overall Passing": average_percent_overall_passing_by_type
})

# Display results
display(type_summary)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.472500,83.897500,93.620000,96.586250,86.900000
District,76.955714,80.965714,66.548571,80.798571,38.804286
